# Script to send CSV File to AWS Kinesis

### Import Packages

In [1]:
import boto3
import csv
import json
import config

### Establish AWS Credentials

In [2]:
aws_access_key_id = config.aws_access_key_id
aws_secret_access_key = config.aws_secret_access_key
region_name = config.region_name
stream_name = config.stream_name

### Initialize Kinesis Client

In [ ]:
kinesis = boto3.client('kinesis', 
                       aws_access_key_id=aws_access_key_id,
                       aws_secret_access_key=aws_secret_access_key,
                       region_name=region_name)

### Define Script Functions to send and read CSV File

In [ ]:
# Function to send records to Kinesis
def send_to_kinesis(data):
    for record in data:
        # Convert each record to JSON format
        json_data = json.dumps(record)
        # Send the record to Kinesis
        response = kinesis.put_record(StreamName=stream_name, Data=json_data, PartitionKey='1')
        print("Record sent to Kinesis: {}".format(response['SequenceNumber']))

# Read CSV file and send records to Kinesis
def send_csv_to_kinesis(file_path):
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        records = []
        for row in csv_reader:
            records.append(row)
            if len(records) == 500:  # Send 500 records at a time
                send_to_kinesis(records)
                records = []
        if records:  # Send remaining records
            send_to_kinesis(records)

### Define Path to Source CSV File

In [ ]:
csv_file_path = '/Users/jsifontes/Documents/Dev/DataSet Creation/Data Cloud/web_engagements.csv'

### Send file to Kinesis

In [ ]:
send_csv_to_kinesis(csv_file_path)
print("CSV file sent to Kinesis Data Stream.")